***ML models for sentimental analysis***
________________________________________________________________________________

Naive bayes is considered one of the best model for text classification and sentimental analysis as it is based on probabilistic approach.

Earlier In svm, dividing boundary is always linear.
It was drawback for non-linear data, so **kernel** was introduced in which transforms feature space so that data can be linearly separable.
Some kernels used are - radial bias kernel, sigmoidal kernel, etc

## Import libraries

In [ ]:
import re           # regex expression
import string       # used for string analysis(or cleaning), string.digits, string.alphabets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

## Download Natural Language Toolkit (nltk) module data

In [ ]:
foo_nltk = __import__('nltk')
foo_nltk.download('stopwords')      # stopwords - words which does not add much meaning to a sentence and safely ignored without sacrificing meaning of sentence
foo_nltk.download('punkt')          # punctuation
foo_nltk.download('averaged_perceptron_tagger')     # arranging lemma according to sentence (identifying part of speech)
foo_nltk.download('tagsets')        # find set of lemma 
foo_nltk.download('treebank')       # forms tree of parts of speech of each word
foo_nltk.download('wordnet')        # website where millions of english word are stored along with all meaning of word, connectivity of words, tell meaning of word in sentemce
foo_nltk.download('twitter_samples')      # dataset
foo_nltk.download('omw-1.4')        # open multi language wordnet

del foo_nltk

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-

## Load data

In [ ]:
from nltk.corpus import twitter_samples
# 20000 samples (5000 each positive and negative)
random_indexes=np.random.randint(0,high=5000,size=5)

**Sample tweets**

In [ ]:
a='😍'
type(a)
# python uses Unicode format for string so smileys can also be used as smileys has unicodes not ascii values

str

In [ ]:
tweets_data=twitter_samples.strings(fileids='tweets.20150430-223406.json')
print('Total number of samples:',len(tweets_data))        # total number of tweets
print()
# see and print random tweets (any 5)
for index in np.random.randint(0,high=len(tweets_data),size=5):
  print('Tweet number:',index,':',tweets_data[index])

Total number of samples: 20000

Tweet number: 7949 : RT @sturdyAlex: A message to #UKIP. 

Because I'm bored of typing the same stuff over and over. 

If you agree, RT. http://t.co/TxT7PKNmqT
Tweet number: 12993 : RT @BBCPropaganda: Even Nick Robinson struggling to spin it as a Miliband win. Cameron must have won by a mile. #BBCNews
Tweet number: 15428 : @andreajenkyns The Tories have failed on all budget targets. Lost AAA rating and borrowed more than all previous Labour Governments #Amnesia
Tweet number: 16771 : Ed Miliband: 'Prefiero no ser primer ministro a pactar con el SNP': El líder laborista ha sido muy claro en la... http://t.co/eG3qNEDVdP
Tweet number: 9980 : RT @Nigel_Farage: Mr Cameron promised a referendum and never delivered. We need #UKIP MPs in the House of Commons: #AskFarage #bbcqt http:/…


Positive tweets

In [ ]:
positive_tweets=twitter_samples.strings(fileids='positive_tweets.json')
print('Total number of samples:',len(positive_tweets))        # total number of positive tweets
print()
# see and print random positive tweets (any 5)
for index in random_indexes:
  print('Tweet number:',index,':',positive_tweets[index])

Total number of samples: 5000

Tweet number: 3629 : @Dodi_Behiri congrats :) exciting
Tweet number: 4888 : gooodnight i love all the boys and iv ris zoe katy jaycy jen :)
Tweet number: 382 : "Chinas actually a girl.
That or a homosexual."
"But we will love him/her either way." 
:D
#HyungBot
Tweet number: 2790 : @stumpedgirl hi there we made our final design, please check it :) https://t.co/EXt098Yq1b
Tweet number: 1236 : @FridaybMaria Hi! Would you like an @imPastel concert? Let me know your city and country and I'll start working on it! Thanks :)


Negative tweets

In [ ]:
negative_tweets=twitter_samples.strings(fileids='negative_tweets.json')
print('Total number of samples:',len(negative_tweets))        # total number of negative tweets
print()
# see and print random negative tweets (any 5)
for index in random_indexes:
  print('Tweet number:',index,':',negative_tweets[index])

Total number of samples: 5000

Tweet number: 3629 : Short weekend ahead :(
Tweet number: 4888 : I apologize for annoying you so much on snapchat :( @JoshDevineDrums
Tweet number: 382 : Agh @UPCIreland keeps cutting out in the middle of #undercoverboss :-(
Tweet number: 2790 : @kaylakerwin :( that disgusts me what a piece of shit..you're such an amazing mother Mateo &amp; Brooklyn are so lucky💗 he doesn't deserve to
Tweet number: 1236 : @Kimberly_171 my poor baby :(


## Data Cleaner functions

In [ ]:
positive_schar=''         # for positive special characters

#find all special characters in positive tweets and add in string defined
for positive_tweet in positive_tweets:
  positive_schar+= re.sub('[a-zA-Z0-9]','',positive_tweet)         # re.sub() finds patterns and replace

# clean special characters (remove spaces, tab spaces or next line charcters)
positive_schar = re.sub('[\t\n ]+','',positive_schar)

In [ ]:
positive_schar

'#@_@@_:)@!:/:)!@:).?!@:)!!!:)@@:)#://./\'!!!-:)://./@,’:)..,:,-.…:.#@@_:)\'........:)-://./://./@@&;://./:)@,\':)!:)://./-:-)\'...:)###.@@:\'@_:))??—:)://./#@_@_@:)@\'.,#\':)@!:)):)&;&;@@💅🏽💋-:))))\'@@@_@!:-)@......:)(é)-:)-():...://./@!@.\':)@.:-#######:)://./@:)))))#_,!:)[]##!@@?:)&;&;:-)@@&;://./:)#@@:):-)@@@@@#:-)@:@@,!!:-)@_.:@.:)@://./:)-/#:)!@:)@@@_,:!@;),.:).@@_,:)#!:)://./#@!:@:)#@_@_@_:)\':)@@,.\'.:-),,.:)@?:)@_@@@@,🍭:).&;:)#://./@:):)@:"@:!:)"@@&;://./:)@__@,!!!:@:)#://./@!\':),?:))@:).:)://./.@\'...:)#@@_@:)#@:),,,…://./:):@@_:—,:,...://./:)://./_!://./.:)@\'..#:-)#@@@_:):):)@:!@@?:)&;&;!#&;@_&;@:://./@:)&;:)))#:@!@://./:)://./:)&;&;@@_@!!:)@@:))://./@://./:)-/#@@@:)@@&;://./:)\':).@,\':)://./:):-)@!:))))))))).:)://./.!:@_@_\'-!!:@:)@:)://./:).:)://./.#@_@@:)@@_:@!!.:)@:)@:-),:)!@..:)@\':):://./#@@@__:)@@_@_@@:).!:-)@💪,:)",:)"\'.!@@?:)&;&;@??!:)!!@@&;://./:)@!@?\'!:),...,!!..;..-:))://./,,,:)://./@@@@_@_!:)@@\':-):)://./#@@@:):)@@@@_@_@@@@@:)@:))@%:)@:@:)@:)#@@@_:)@,:).:-)@

In [ ]:
negative_schar=''         # for negative special characters

#find all special characters in negative tweets and add in string defined
for negative_tweet in negative_tweets:
  negative_schar+= re.sub('[a-zA-Z0-9]','',negative_tweet)         # re.sub() finds patterns and replace

# clean special characters (remove spaces, tab spaces or next line charcters)
negative_schar = re.sub('[\t\n ]+','',negative_schar)

In [ ]:
negative_schar

':(.\':(@.:(“@:"":(:(”"":(,\':(://./@:((@@.:(://./?:("@::(://./"#:-(#######…://./&;\':(((@:(@,.:*:(\':(@&;.:-(."\'""-".........:(@_:(@:(\'\':(.:(://./:(@@@@@,.:(?😩/:(:(@__?\':(://./:-(:(://./@:(\'@:(.:(@:(.’@.:-(’#@_:(?@@@:(.:($$$$:(😭😭😭😭😭😭://./@@(,\').:.:-(\':((@.?:(!!!\':((@\':(@:(!:(@.:(:(("@_:@_,\'.."´:´(—-\'....:((@\':(@-.:-(@\'😩?:-(@\'?:(@@__@@@@:(::(:(://./..:-(##\'.:...://./##:-(@:(##--...://./##:-(##,...://./##:-(##\'\'\'\'\'...://./##:-(##\'\'.:...://./##:-(@:(@:(((((((((:(:-(:((:-(\'@:(:-(@\',:(@:(#://./@\':(!??:(:(@_\':(:(@:(:(:(://./:-(://./:-(@\'\'.:(@:(:-(:((((@:(:((((#:(:(@:(💜..\'!!:-(://./@??:(((:(:(:((((@\':((/):(:(:(#:(@。。。_:(´ω`」∠):_@_💕:(!?😊":-("\':(@@_.\'.:(@:((@.\'...:(:(-#######:(://./:(@\':(@:(((@___\':((@:((@_:(:(....://./:(@@\':-(@:(((((((((((((((@:-(:(@_?:(:(:(((((,:-(://./@_:((:(((:((((:(!:(@:(\'@,:(?\'?:(@:(\':(:((((@:(\':(:(@_:(.@:(@_?:(.:(:-(@_:(😂😩@-!\'!!:(:(:(@:((@:(@:(@\'!&;:(://./,,:((@://./:(@@\':((@:(@@:-(\'.&;.@:-(((((@:(@@_&;:(://./:(://./\':((:(@:(

In [ ]:
print(':) in positive tweets:',positive_schar.count(':)'))
print(':) in negative tweets:',negative_schar.count(':)'))

:) in positive tweets: 3746
:) in negative tweets: 8


In [ ]:
from nltk.tag import pos_tag        # load sentence tagger
from nltk.stem.snowball import SnowballStemmer          # remove -s,-es,-ing etc suffuxes from word
from nltk.stem.wordnet import WordNetLemmatizer         # find root word, it need to know part of speech of word in order to find root word

#function to clean grammar
def clean_grammer(value:list,
                  stemmer_=SnowballStemmer('english',ignore_stopwords=False),
                  lemmatizer_=WordNetLemmatizer()):
  # Valid options for parts of speech in WordNetLemmatizer are 'n' for nouns, 'v' for verbs, 'a' for adjectives, 'r' for adverbs and 's' for satellite adjectives
  """
  POS tag - meaning -> lemmatizer 

  J - adjective -> a
  N - noun -> n
  R - adverb -> r
  V - verb -> v
  """

  clean_value=list()        # list of clean sentences

  for word,pos in pos_tag(value):
    if stemmer_:word=stemmer_.stem(word)              # apply stemmer
    elif pos.startwith('J'):word=lemmatizer_.lemmatize(word,'a')
    elif pos.startwith('R'):word=lemmatizer_.lemmatize(word,'r')
    elif pos.startwith('V'):word=lemmatizer_.lemmatize(word,'v')
    # elif pos.startwith('N'):word=lemmatizer_.lemmatize(word,'n')
    else: word=lemmatizer_.lemmatize(word,'n')

    clean_value.append(word)          # append clean words to list of clean words
  
  return clean_value

In [ ]:
from nltk.tokenize import word_tokenize           # load word tokenize
from nltk.corpus import stopwords                 # load stopwords(helping words or words that don't add meaning to sentence)

def clean(value:str,
          tags2remove:list=['http[\S]+','@[\S]+','&amp[\S]+'],            # \S - string 
          tag2convert:dict={' positive ':[':)',';)',':-)',':p',':D'],
                            ' negative ':[':(',';(',':-(']},
          cleaner:str='[^a-zA-Z ]',
          stopwords:list=stopwords.words(fileids='english'),
          minimum_word_length:int=3,
          call_clean_grammer:bool=True,
          clean_grammer_method=clean_grammer):
  
  # if value is list or tuple
  if isinstance(value,(list,tuple)):
    # call clean itself
    value=[clean(sentc) for sentc in value]
    return value          # return cleaned value

  #remove links and @ tags
  for tag in tags2remove:
    value=re.sub(tag,'',value)        # remove and update string
  
  # replace special character tags
  for tag_name,tags in tag2convert.items():
    for tag in tags:
      value=value.replace(tag,tag_name)       # replace tag woth positive or negative

  value=re.sub(cleaner,'',value)              # remove all punctuations

  value=value.casefold()                      # casefold() - converts to lower case forcefully

  value=word_tokenize(value)                  # break sentence to list of words (string -> list)

  value=[word for word in value if word not in stopwords]             # remove stopwords

  value=[word for word in value if not len(word)<minimum_word_length]            # remove words less than given minimum word length

  if call_clean_grammer: return clean_grammer_method(value)
  else: return value              #return clean string without cleaning grammer

## Data Pre-processing functions

In [ ]:
# function to get modeled dictionary like [python,language] -> {'python':True,'language':True}
def tokenizeSentenceGen(data:list)->dict:           # Generator function (always contains yield sentence)
  #data : list of sentences (each having list of words)
  for sentence in data:
    yield dict([(word,True) for word in sentence])      # make yield sentemce as dictionary - it remembers what it previously returned

def dataGenerator(data:list):
  #data : list of sentences (each having list of words)

  for sentence in data:             # to get tokenized sentence from data
    for token in sentence:          # to get word from sentence
      yield token                   

## Clean Data

In [ ]:
#print any 5 random positive tweets
for index in random_indexes:
  print('Tweet number:',index,':',positive_tweets[index])

Tweet number: 3629 : @Dodi_Behiri congrats :) exciting
Tweet number: 4888 : gooodnight i love all the boys and iv ris zoe katy jaycy jen :)
Tweet number: 382 : "Chinas actually a girl.
That or a homosexual."
"But we will love him/her either way." 
:D
#HyungBot
Tweet number: 2790 : @stumpedgirl hi there we made our final design, please check it :) https://t.co/EXt098Yq1b
Tweet number: 1236 : @FridaybMaria Hi! Would you like an @imPastel concert? Let me know your city and country and I'll start working on it! Thanks :)


In [ ]:
#cleaning positive tweets
clean_tokenized_positive_tweets=clean(positive_tweets)
for index in random_indexes:
  print('Tweet number:',index,':',clean_tokenized_positive_tweets[index])

Tweet number: 3629 : ['congrat', 'posit', 'excit']
Tweet number: 4888 : ['gooodnight', 'love', 'boy', 'ris', 'zoe', 'kati', 'jayci', 'jen', 'posit']
Tweet number: 382 : ['china', 'actual', 'girlthat', 'homosexualbut', 'love', 'himher', 'either', 'way', 'posit', 'hyungbot']
Tweet number: 2790 : ['made', 'final', 'design', 'pleas', 'check', 'posit']
Tweet number: 1236 : ['would', 'like', 'concert', 'let', 'know', 'citi', 'countri', 'ill', 'start', 'work', 'thank', 'posit']


In [ ]:
#print any 5 random negative tweets
for index in random_indexes:
  print('Tweet number:',index,':',negative_tweets[index])

Tweet number: 3629 : Short weekend ahead :(
Tweet number: 4888 : I apologize for annoying you so much on snapchat :( @JoshDevineDrums
Tweet number: 382 : Agh @UPCIreland keeps cutting out in the middle of #undercoverboss :-(
Tweet number: 2790 : @kaylakerwin :( that disgusts me what a piece of shit..you're such an amazing mother Mateo &amp; Brooklyn are so lucky💗 he doesn't deserve to
Tweet number: 1236 : @Kimberly_171 my poor baby :(


In [ ]:
#cleaning positive tweets
clean_tokenized_negative_tweets=clean(negative_tweets)
for index in random_indexes:
  print('Tweet number:',index,':',clean_tokenized_negative_tweets[index])

Tweet number: 3629 : ['short', 'weekend', 'ahead', 'negat']
Tweet number: 4888 : ['apolog', 'annoy', 'much', 'snapchat', 'negat']
Tweet number: 382 : ['agh', 'keep', 'cut', 'middl', 'undercoverboss', 'negat']
Tweet number: 2790 : ['negat', 'disgust', 'piec', 'shityour', 'amaz', 'mother', 'mateo', 'brooklyn', 'lucki', 'doesnt', 'deserv']
Tweet number: 1236 : ['poor', 'babi', 'negat']


## Prepare data

In [ ]:
positiveGenerator=dataGenerator(clean_tokenized_positive_tweets)
print(f'Type of positive_word_gen_ -> {positiveGenerator}:', type(positiveGenerator))
print()

# make frequency dict
#from nltk import FreqDist
from nltk.probability import FreqDist

# make positive words frequency dictionary
positive_words_frequency=FreqDist(positiveGenerator)
print(positive_words_frequency)
print()

print('Frequency of top 10 most common elements[positive]:\n', positive_words_frequency.most_common(10),sep='')
print()

print('Type of positive_words_frequency ->',type(positive_words_frequency))

Type of positive_word_gen_ -> <generator object dataGenerator at 0x7fed70333510>: <class 'generator'>

<FreqDist with 6134 samples and 31119 outcomes>

Frequency of top 10 most common elements[positive]:
[('posit', 5239), ('thank', 638), ('follow', 440), ('love', 399), ('day', 243), ('good', 235), ('like', 231), ('happi', 210), ('get', 209), ('see', 184)]

Type of positive_words_frequency -> <class 'nltk.probability.FreqDist'>


In [ ]:
negativeGenerator=dataGenerator(clean_tokenized_negative_tweets)
print(f'Type of positive_word_gen_ -> {negativeGenerator}:', type(negativeGenerator))
print()

# make frequency dict
#from nltk import FreqDist
from nltk.probability import FreqDist

# make positive words frequency dictionary
negative_words_frequency=FreqDist(negativeGenerator)
print(negative_words_frequency)
print()

print('Frequency of top 10 most common elements[negative]:\n', negative_words_frequency.most_common(10),sep='')
print()

print('Type of negative_words_frequency ->',type(negative_words_frequency))

Type of positive_word_gen_ -> <generator object dataGenerator at 0x7fed703335f0>: <class 'generator'>

<FreqDist with 5920 samples and 30201 outcomes>

Frequency of top 10 most common elements[negative]:
[('negat', 5149), ('miss', 300), ('want', 244), ('pleas', 237), ('get', 233), ('dont', 232), ('cant', 230), ('follow', 228), ('like', 221), ('time', 166)]

Type of negative_words_frequency -> <class 'nltk.probability.FreqDist'>


Make dataset for model

In [ ]:
positive_words4Data=tokenizeSentenceGen(clean_tokenized_positive_tweets)
negative_words4Data=tokenizeSentenceGen(clean_tokenized_negative_tweets)

# print first sample
print('Sample of positive tweets:', next(tokenizeSentenceGen(clean_tokenized_positive_tweets)))

Sample of positive tweets: {'followfriday': True, 'top': True, 'engag': True, 'member': True, 'communiti': True, 'week': True, 'posit': True}


In [ ]:
# make positive and negative dataset
positive_dataset=[(tweet_dict,'Positive') for tweet_dict in positive_words4Data]
negative_dataset=[(tweet_dict,'Negative') for tweet_dict in negative_words4Data]

# merge dataset
dataset=positive_dataset+negative_dataset
print('Sixe of dataset:',len(dataset))

Sixe of dataset: 10000


Split dataset

In [ ]:
from sklearn.model_selection import train_test_split
train_set,test_set=train_test_split(dataset,test_size=0.10)

print(f'Size of train dataset: {len(train_set)}')
print(f'Size of test dataset: {len(test_set)}')

Size of train dataset: 9000
Size of test dataset: 1000


In [ ]:
# print first sample (final data)
[(tweet_dict,'Positive') for tweet_dict in tokenizeSentenceGen(clean_tokenized_positive_tweets)][0]

({'followfriday': True,
  'top': True,
  'engag': True,
  'member': True,
  'communiti': True,
  'week': True,
  'posit': True},
 'Positive')

## Applying Machine Learning Model

In [ ]:
from nltk import classify         # to compute accuracy
from nltk import NaiveBayesClassifier

classifier_=NaiveBayesClassifier.train(train_set)           # train model
print('Accuracy score:', round(classify.accuracy(classifier_,test_set)*100,3))          # test model

Accuracy score: 99.6


In [ ]:
# classifier 10 most informative features
classifier_.show_most_informative_features(10)

Most Informative Features
                   negat = True           Negati : Positi =   1789.3 : 1.0
                   posit = True           Positi : Negati =    990.6 : 1.0
                   posit = None           Negati : Positi =    134.5 : 1.0
                   negat = None           Positi : Negati =    119.7 : 1.0
                     bam = True           Positi : Negati =     26.9 : 1.0
                     sad = True           Negati : Positi =     26.0 : 1.0
               communiti = True           Positi : Negati =     20.9 : 1.0
                   arriv = True           Positi : Negati =     17.2 : 1.0
               goodnight = True           Positi : Negati =     14.9 : 1.0
                  welcom = True           Positi : Negati =     14.3 : 1.0


## Test on other data

In [ ]:
tweet='''It is with immeasurable grief that we confirm the passing of Chadwick Boseman. Chadwick was diagnosed with stage III colon cancer in 2016, 
and battled with it these last 4 years as it progressed to stage IV...
'''
classifier_.classify(dict([(word,True) for word in clean(tweet)]))

'Negative'

In [ ]:
tweet2='''yes, please do enlighten me. email me at smalldickenergy@getalife.com '''
classifier_.classify(dict([(word,True) for word in clean(tweet2)]))

'Positive'

In [ ]:
tweet3='''I hope that even my worst critics remain on Twitter, because that is what free speech means '''
classifier_.classify(dict([(word,True) for word in clean(tweet3)]))

'Negative'

In [ ]:
tweet4='''Congratulations to the Astronauts that left Earth today. Good choice '''
classifier_.classify(dict([(word,True) for word in clean(tweet4)]))

'Positive'

In [ ]:
tweet5='''😙'''
classifier_.classify(dict([(word,True) for word in clean(tweet5)]))

'Positive'

In [ ]:
check=''' Best wishes to Minister Smt. 
@smritiirani
 Ji on her birthday. She is providing admirable leadership to our Government's efforts to further the welfare of our Nari Shakti. 
 She is admired for her oratory and connect with people from all walks of life. Praying for her long life.'''

classifier_.classify(dict([(word,True) for word in clean(check)]))

'Negative'

## Analysing hindi tweets

In [ ]:
!python -m pip install -qq translate

In [ ]:
# load module translator
from translate import Translator

translator=Translator(to_lang='en',from_lang='hi')

#make translation
translation=translator.translate("यह एक पायथन भाषा है|")
print(f'"यह एक पायथन भाषा है|" -> {translation}')

"यह एक पायथन भाषा है|" -> It's a Python language.


In [ ]:
tweet6='''हमारे देश और दुनिया भर में कई पत्रकार कठिन और प्रतिकूल परिस्थितियों में काम करते हैं, लेकिन वो किसी भी विपत्ति और विरोध के सामने डटे रहते हैं. ये वो गुण है, जिसे खोना नहीं चाहिए.'''

translator=Translator(to_lang='en',from_lang='hi')

classifier_.classify(dict([(word,True) for word in clean(translator.translate(tweet6))]))

'Negative'

In [ ]:
tweet7='''मंत्री श्रीमती जी को हार्दिक शुभकामनाएं
@smritiirani
  जी के जन्मदिन पर। वह हमारी नारी शक्ति के कल्याण के लिए हमारी सरकार के प्रयासों को सराहनीय नेतृत्व प्रदान कर रही हैं। 
  उनकी वक्तृत्व कला के लिए उनकी प्रशंसा की जाती है और वे जीवन के सभी क्षेत्रों के लोगों से जुड़ती हैं। उनकी लंबी उम्र की दुआ कर रहे हैं.'''

translator=Translator(to_lang='en',from_lang='hi')
classifier_.classify(dict([(word,True) for word in clean(translator.translate(tweet7))]))

'Negative'

## Save and Deploy model

In [ ]:
import pickle

with open('nltk.nb.model',mode='wb') as model_file:           # nltk.nb - model name
  pickle.dump(classifier_,model_file)